In [1]:
import sys, os
import numpy as np
from sklearn.model_selection import LeaveOneOut

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from dependencies.mgeval import core, utils
from src.evaluation import mgeval, evaluation

C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
test_set_of_sequences = [{'note_count': 4, 'pitch_count': 7, 'avg_ioi': 20}, {'note_count': 4, 'pitch_count': 5, 'avg_ioi': 10}, {'note_count': 4, 'pitch_count': 3, 'avg_ioi': 0}]

In [3]:
result = {}
for dictionary in test_set_of_sequences:
    for key, value in dictionary.items():
        if key not in result.keys():
            result[key] = []
        result[key].append(value)

In [4]:
set_of_sequences = result
max_key = max(set_of_sequences, key= lambda x: len(set(set_of_sequences[x])))
set_of_sequences

{'note_count': [4, 4, 4], 'pitch_count': [7, 5, 3], 'avg_ioi': [20, 10, 0]}

In [5]:
num_samples = len(set_of_sequences[max_key])
num_metrics = len(set_of_sequences)
print('size:', num_samples, num_metrics)

loo = LeaveOneOut()
loo.get_n_splits(np.arange(num_samples))
intra_set_distances = np.zeros((num_samples, num_metrics, num_samples-1))
# for i in range(len(set_of_sequences)):
i = 0
for key in set_of_sequences:
    for train_indexes, test_index in loo.split(np.arange(num_samples)):
        # x = utils.c_dist(set_of_sequences[(int)(test_index)][key], [ set_of_sequences[idx][key] for idx in train_indexes ])
        # train_index = np.array(train_indexes)[indices.astype(int)]
        x = utils.c_dist(np.array(set_of_sequences[key])[test_index], np.array(set_of_sequences[key])[train_indexes])
        intra_set_distances[test_index[0]][i] = x
        print(test_index, i, x)
    i += 1
intra_set_distances

size: 3 3
[0] 0 [0. 0.]
[1] 0 [0. 0.]
[2] 0 [0. 0.]
[0] 1 [2. 4.]
[1] 1 [2. 2.]
[2] 1 [4. 2.]
[0] 2 [10. 20.]
[1] 2 [10. 10.]
[2] 2 [20. 10.]


array([[[ 0.,  0.],
        [ 2.,  4.],
        [10., 20.]],

       [[ 0.,  0.],
        [ 2.,  2.],
        [10., 10.]],

       [[ 0.,  0.],
        [ 4.,  2.],
        [20., 10.]]])

In [6]:
distances = mgeval.calc_intra_set_distances(test_set_of_sequences)
distances

array([[[ 0.,  0.],
        [ 2.,  4.],
        [10., 20.]],

       [[ 0.,  0.],
        [ 2.,  2.],
        [10., 10.]],

       [[ 0.,  0.],
        [ 4.,  2.],
        [20., 10.]]])

In [7]:
distances[0][0]

array([0., 0.])

In [8]:
# ATTENTION - This test doesn't work anymore, because evaluate_variance expects a list of PM to be analysed

e = evaluation.Evaluation()
v = e.evaluate_variance(test_set_of_sequences)
v

[EVAL] Error: No Normalization Factors set.


{'absolute': {'note_count': 0.0,
  'pitch_count': 2.6666666666666665,
  'avg_ioi': 13.333333333333334},
 'normalized': None}